# 競走表（K）データ取得

In [26]:
# 時間を制御する time モジュールをインポート
from time import sleep

# HTTP通信ライブラリの requests モジュールをインポート
from requests import get

# 日付を扱うための datetime モジュールをインポート
from datetime import datetime as dt
from datetime import timedelta as td

# フォルダ(ディレクトリ)を作成する makedirs モジュールをインポート
from os import makedirs

# 開始日と終了日を指定(YYYY-MM-DD)
START_DATE = "2024-11-01"
END_DATE = "2024-11-01"

# ファイルの保存先を指定　
SAVE_DIR = "results_lzh/"

# リクエスト間隔を指定(秒)　※サーバに負荷をかけないよう3秒以上を推奨
INTERVAL = 3

# URLの固定部分を指定
FIXED_URL = "http://www1.mbrace.or.jp/od2/K/"

# 開始合図
print("作業を開始します")

# ファイルを格納するフォルダを作成
makedirs(SAVE_DIR, exist_ok=True)

# 開始日と終了日を日付型に変換して格納
start_date = dt.strptime(START_DATE, '%Y-%m-%d')
end_date = dt.strptime(END_DATE, '%Y-%m-%d')

# 日付の差から期間を計算
days_num = (end_date - start_date).days + 1

# 日付リストを格納する変数
date_list = []

# 日付リストを生成
for i in range(days_num):
    # 開始日から日付を順に取得
    target_date = start_date + td(days=i)

    # 日付型を文字列に変換してリストに格納(YYYYMMDD)
    date_list.append(target_date.strftime("%Y%m%d"))

# URL生成とダウンロード
for date in date_list:

    # URL生成
    yyyymm = date[0:4] + date[4:6]
    yymmdd = date[2:4] + date[4:6] + date[6:8]

    variable_url = FIXED_URL + yyyymm + "/k" + yymmdd + ".lzh"
    file_name = "k" + yymmdd + ".lzh"

    # ダウンロード
    r = get(variable_url)

    # 成功した場合
    if r.status_code == 200:
        f = open(SAVE_DIR + file_name, 'wb')
        f.write(r.content)
        f.close()
        print(variable_url + " をダウンロードしました")

    # 失敗した場合
    else:
        print(variable_url + " のダウンロードに失敗しました")

    # 指定した間隔をあける
    sleep(INTERVAL)

# 終了合図
print("作業を終了しました")

作業を開始します
http://www1.mbrace.or.jp/od2/K/202411/k241101.lzh をダウンロードしました
作業を終了しました


## Kデータを解凍

In [27]:
# 正規表現をサポートするモジュール re をインポート
import re
# LZH形式のファイルを解凍するパッケージ lhafile をインポート
import lhafile
# オペレーティングシステムの機能を利用するパッケージ os をインポート
import os

# ダウンロードしたLZHファイルが保存されている場所を指定
LZH_FILE_DIR = "results_lzh"

# 解凍したファイルを保存する場所を指定
TXT_FILE_DIR = "results_txt"

# 開始合図
print("作業を開始します")

# ファイルを格納するフォルダを作成
os.makedirs(TXT_FILE_DIR, exist_ok=True)

# LZHファイルのリストを取得
lzh_file_list = os.listdir(LZH_FILE_DIR)

# 正規表現パターンを定義（日付が含まれる部分にマッチするパターン）
pattern = r'[bk]\d{6}\.lzh'

# ファイルの数だけ処理を繰り返す
for lzh_file_name in lzh_file_list:

    # 正規表現パターンにマッチする場合にのみ実行
    if re.match(pattern, lzh_file_name):

        lzh_file_path = os.path.join(LZH_FILE_DIR, lzh_file_name) # フルパスを作成

        file = lhafile.Lhafile(os.path.join(LZH_FILE_DIR, lzh_file_name))

        # 解凍したファイルの名前を取得
        info = file.infolist()
        name = info[0].filename

        # 解凍したファイルの保存
        open(os.path.join(TXT_FILE_DIR, name), "wb").write(file.read(name))

        print(TXT_FILE_DIR + name + " を解凍しました")

        # LZHファイルを削除
        os.remove(lzh_file_path)
        print(f"{lzh_file_name} を削除しました")

# 終了合図
print("作業を終了しました")

作業を開始します
results_txtK241101.TXT を解凍しました
k241101.lzh を削除しました
作業を終了しました


### Kデータを移動

In [28]:
import os
print(os.getcwd())

/home/jovyan/work/Data_scraping


In [29]:
import os

# 移動元と移動先のディレクトリパス
source_dir = "/home/jovyan/work/Data_scraping/results_txt"
destination_dir = "/home/jovyan/work/train_data/K"

# 移動元のディレクトリ内のファイル一覧を取得
for filename in os.listdir(source_dir):
    # ファイルのパスを結合
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)

    # ファイルであるか確認 (ディレクトリは除外)
    if os.path.isfile(source_path):
        # ファイルを移動 (ファイル名が重複する場合は上書きされる)
        os.rename(source_path, destination_path)
        print(f"Moved: {filename} to {destination_dir}")

print("File transfer completed.")

Moved: K241101.TXT to /home/jovyan/work/train_data/K
File transfer completed.


# 番組表（B）データ取得

In [30]:
# 開始日と終了日を指定(YYYY-MM-DD)
START_DATE = "2024-11-01"
END_DATE = "2024-11-01"

# ファイルの保存先を指定　
SAVE_DIR = "results_lzh/"

# リクエスト間隔を指定(秒)　※サーバに負荷をかけないよう3秒以上を推奨
INTERVAL = 3

# URLの固定部分を指定
FIXED_URL = "http://www1.mbrace.or.jp/od2/B/"


# 開始合図
print("作業を開始します")

# ファイルを格納するフォルダを作成
makedirs(SAVE_DIR, exist_ok=True)

# 開始日と終了日を日付型に変換して格納
start_date = dt.strptime(START_DATE, '%Y-%m-%d')
end_date = dt.strptime(END_DATE, '%Y-%m-%d')

# 日付の差から期間を計算
days_num = (end_date - start_date).days + 1

# 日付リストを格納する変数
date_list = []

# 日付リストを生成
for i in range(days_num):
    # 開始日から日付を順に取得
    target_date = start_date + td(days=i)

    # 日付型を文字列に変換してリストに格納(YYYYMMDD)
    date_list.append(target_date.strftime("%Y%m%d"))

# URL生成とダウンロード
for date in date_list:

    # URL生成
    yyyymm = date[0:4] + date[4:6]
    yymmdd = date[2:4] + date[4:6] + date[6:8]

    variable_url = FIXED_URL + yyyymm + "/b" + yymmdd + ".lzh"
    file_name = "b" + yymmdd + ".lzh"

    # ダウンロード
    r = get(variable_url)

    # 成功した場合
    if r.status_code == 200:
        f = open(SAVE_DIR + file_name, 'wb')
        f.write(r.content)
        f.close()
        print(variable_url + " をダウンロードしました")

    # 失敗した場合
    else:
        print(variable_url + " のダウンロードに失敗しました")

    # 指定した間隔をあける
    sleep(INTERVAL)

# 終了合図
print("作業を終了しました")

作業を開始します
http://www1.mbrace.or.jp/od2/B/202411/b241101.lzh をダウンロードしました
作業を終了しました


## Bデータ解凍

In [31]:
# 正規表現をサポートするモジュール re をインポート
import re
# LZH形式のファイルを解凍するパッケージ lhafile をインポート
import lhafile
# オペレーティングシステムの機能を利用するパッケージ os をインポート
import os

# ダウンロードしたLZHファイルが保存されている場所を指定
LZH_FILE_DIR = "results_lzh"

# 解凍したファイルを保存する場所を指定
TXT_FILE_DIR = "results_txt"

# 開始合図
print("作業を開始します")

# ファイルを格納するフォルダを作成
os.makedirs(TXT_FILE_DIR, exist_ok=True)

# LZHファイルのリストを取得
lzh_file_list = os.listdir(LZH_FILE_DIR)

# 正規表現パターンを定義（日付が含まれる部分にマッチするパターン）
pattern = r'[bk]\d{6}\.lzh'

# ファイルの数だけ処理を繰り返す
for lzh_file_name in lzh_file_list:

    # 正規表現パターンにマッチする場合にのみ実行
    if re.match(pattern, lzh_file_name):

        lzh_file_path = os.path.join(LZH_FILE_DIR, lzh_file_name) # フルパスを作成

        file = lhafile.Lhafile(os.path.join(LZH_FILE_DIR, lzh_file_name))

        # 解凍したファイルの名前を取得
        info = file.infolist()
        name = info[0].filename

        # 解凍したファイルの保存
        open(os.path.join(TXT_FILE_DIR, name), "wb").write(file.read(name))

        print(TXT_FILE_DIR + name + " を解凍しました")

        # LZHファイルを削除
        os.remove(lzh_file_path)
        print(f"{lzh_file_name} を削除しました")

# 終了合図
print("作業を終了しました")

作業を開始します
results_txtB241101.TXT を解凍しました
作業を終了しました


### Bデータを移動

In [32]:
import os

# 移動元と移動先のディレクトリパス
source_dir = "/home/jovyan/work/Data_scraping/results_txt"
destination_dir = "/home/jovyan/work/train_data/B"

# 移動元のディレクトリ内のファイル一覧を取得
for filename in os.listdir(source_dir):
    # ファイルのパスを結合
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)

    # ファイルであるか確認 (ディレクトリは除外)
    if os.path.isfile(source_path):
        # ファイルを移動 (ファイル名が重複する場合は上書きされる)
        os.rename(source_path, destination_path)
        print(f"Moved: {filename} to {destination_dir}")

print("File transfer completed.")

Moved: B241101.TXT to /home/jovyan/work/train_data/B
File transfer completed.
